# LangChain's components

## Models and Prompts

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAI

# Load environment variables
load_dotenv()

# Initialize the OpenAI language model
llm = OpenAI()

# Use the invoke method instead of direct calling
print(llm.invoke('tell me a joke'))

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

# Create the prompt template
template = """Translate the following English sentence to {language}:
Sentence: {sentence}
Translation:"""
prompt = PromptTemplate.from_template(template)

# Initialize the language model
llm = OpenAI()

# Format the prompt and get the translation
formatted_prompt = prompt.format(sentence="the cat is on the table", language="spanish")
translation = llm.invoke(formatted_prompt)
print(translation)

## Data Connections

### Document loaders

In [ ]:
import csv

# Sample data
data = [
    ['Name', 'Age', 'City'],
    ['John', 25, 'New York'],
    ['Emily', 28, 'Los Angeles'],
    ['Michael', 22, 'Chicago']
]

# File name
file_name = 'sample.csv'

# Write data to CSV file
with open(file_name, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

print(f'Sample CSV file "{file_name}" generated and saved.')



In [ ]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(file_path='sample.csv')
data = loader.load()
print(data)

### Document splitters

In [ ]:
# Sample sentences about mountains and nature
content = """Amidst the serene landscape, towering mountains stand as majestic guardians of nature's beauty.
The crisp mountain air carries whispers of tranquility, while the rustling leaves compose a symphony of wilderness.
Nature's palette paints the mountains with hues of green and brown, creating an awe-inspiring sight to behold.
As the sun rises, it casts a golden glow on the mountain peaks, illuminating a world untouched and wild."""

# File name
file_name = 'mountain.txt'

# Write content to text file
with open(file_name, 'w') as txtfile:
    txtfile.write(content)

# Read the file
with open('mountain.txt') as f:
    mountain = f.read()

# Import from langchain.text_splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

# Split the text
texts = text_splitter.create_documents([mountain])

# Print each text chunk
print(texts[0])
print(texts[1])
print(texts[2])

### Text embedding models

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize embeddings model
embeddings_model = OpenAIEmbeddings(model='text-embedding-ada-002')

# Embed documents
embeddings = embeddings_model.embed_documents(
    [
        "Good morning!",
        "Oh, hello!",
        "I want to report an accident",
        "Sorry to hear that. May I ask your name?",
        "Sure, Mario Rossi."
    ]
)

# Print document embeddings information
print("Embed documents:")
print(f"Number of vector: {len(embeddings)}; Dimension of each vector: {len(embeddings[0])}")

# Embed query
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

# Print query embedding information
print("Embed query:")
print(f"Dimension of the vector: {len(embedded_query)}")
print(f"Sample of the first 5 elements of the vector: {embedded_query[:5]}")

In [ ]:
#saving the conversation in a txt file
# List of dialogue lines
dialogue_lines = [
    "Good morning!",
    "Oh, hello!",
    "I want to report an accident",
    "Sorry to hear that. May I ask your name?",
    "Sure, Mario Rossi."
]

# File name
file_name = 'dialogue.txt'

# Write dialogue lines to text file
with open(file_name, 'w') as txtfile:
    for line in dialogue_lines:
        txtfile.write(line + '\n')

print(f'Dialogue text file "{file_name}" generated and saved.')


### Vector stores

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]

# Load the document, split it into chunks, embed each chunk and load it into the vector store.

raw_documents = TextLoader('dialogue.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())


In [ ]:
query = "What is the reason for calling?"
docs = db.similarity_search(query)
print(docs[0].page_content)

In [ ]:
print(documents[2])

### Retrievers

In [ ]:
from langchain_openai import OpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Assuming 'db' is already defined from a previous vector store creation
retriever = db.as_retriever()

In [ ]:
# Create the QA chain with updated imports and initialization
qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | PromptTemplate.from_template(
        "Given the following context, answer the question:\n\n{context}\n\nQuestion: {question}\nAnswer:"
    )
    | OpenAI()
    | StrOutputParser()
)

# Run the query
query = "What was the reason of the call?"
result = qa_chain.invoke(query)
print(result)

## Memory

In [ ]:
from langchain_openai import OpenAI
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory

# Initialize the OpenAI language model
llm = OpenAI(temperature=0)

# Create memory with the OpenAI model
memory = ConversationSummaryMemory(llm=llm)

# Save context to the memory
memory.save_context(
    {"input": "hi, I'm looking for some ideas to write an essay in AI"}, 
    {"output": "hello, what about writing on LLMs?"}
)

# Load memory variables
memory_vars = memory.load_memory_variables({})
print(memory_vars)

In [ ]:
ConversationSummaryMemory.save_context?

## Chains

### Simple Chain

In [ ]:
from langchain_openai import OpenAI
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory

# Initialize the OpenAI language model
llm = OpenAI(temperature=0)

# Create memory with the OpenAI model
memory = ConversationSummaryMemory(llm=llm)

# Save context to the memory
memory.save_context(
    {"input": "hi, I'm looking for some ideas to write an essay in AI"}, 
    {"output": "hello, what about writing on LLMs?"}
)

# Load memory variables
memory_vars = memory.load_memory_variables({})
print(memory_vars)

### Router chain

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# Initialize the OpenAI language model
llm = OpenAI()

# Define templates
itinerary_template = """You are a vacation itinerary assistant. \
You help customers finding the best destinations and itinerary. \
You help customer screating an optimized itinerary based on their preferences.
Here is a question:
{input}"""

restaurant_template = """You are a restaurant booking assitant. \
You check with customers number of guests and food preferences. \
You pay attention whether there are special conditions to take into account.
Here is a question:
{input}"""

# Prompt information
prompt_infos = [
    {
        "name": "itinerary",
        "description": "Good for creating itinerary",
        "prompt_template": itinerary_template,
    },
    {
        "name": "restaurant",
        "description": "Good for help customers booking at restaurant",
        "prompt_template": restaurant_template,
    },
]

# Create destination chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# Create default chain
default_chain = ConversationChain(llm=llm, output_key="text")

# Prepare router destinations
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

# Create router prompt and chain
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# Create multi-prompt chain
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [ ]:
print(chain.invoke("I'm planning a trip from Milan to Venice by car. What can I visit in between?"))

In [ ]:
print(chain.invoke("I want to book a table for tonight"))

### Sequential Chain

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Initialize the OpenAI language model
llm = OpenAI(temperature=0)

# Joke generation chain
template = """You are a comedian. Generate a joke on the following {topic}
Joke:"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
joke_chain = LLMChain(llm=llm, prompt=prompt_template)

# Translation chain
template = """You are translator. Given a text input, translate it to {language}
Translation:"""
prompt_template = PromptTemplate(input_variables=["language"], template=template)
translator_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True)
translated_joke = overall_chain.run("Cats and Dogs")

### Transformation chain

In [ ]:
# Import the string module
import string

# Define the function
def rename_cat(inputs: dict) -> dict:
  # Open the file in read mode
  text = inputs["text"]
  # Create a table that maps punctuation characters to None
  new_text = text.replace('cat', 'Silvester the Cat')
  # Apply the table to the text and return the result
  return {"output_text": new_text}




In [ ]:
from langchain_openai import OpenAI
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

# Read the file
with open("Cats&Dogs.txt") as f:
    cats_and_dogs = f.read()

import string

# Define the rename_cat function (which was missing in the original code)
def rename_cat(inputs):
    text = inputs['text']
    # Example transformation - replace 'cat' with 'feline'
    return {'output_text': text.replace('cat', 'feline')}

# Create transform chain
transform_chain = TransformChain(
    input_variables=["text"], 
    output_variables=["output_text"], 
    transform=rename_cat
)

# Create prompt template
template = """Summarize this text:
{output_text}
Summary:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)

# Create LLM chain
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)

# Create sequential chain
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

# Run the chain
result = sequential_chain.run(cats_and_dogs)
print(result)

## Agents

In [ ]:
from langchain_community.utilities.serpapi import SerpAPIWrapper
from langchain_openai import OpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain_core.tools import Tool  # Updated import for Tool
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Make sure OpenAI API key is set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY environment variable is not set")

# Initialize search and OpenAI
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)  # Added temperature parameter for more factual responses

# Define tools
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

# Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True  # Added error handling
)

try:
    # Run the agent
    result = agent.invoke({"input": "When was Avatar 2 released?"})  # Updated to invoke() method
    print(result["output"])
except Exception as e:
    print(f"An error occurred: {e}")

# Start working with LLMs in Hugging Face Hub

In [ ]:
#!pip install python-dotenv   #installing the required package
#!pip install huggingface_hub

#option 1: get your tokens from the .env file

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"]


In [ ]:
#option 2: get the token with the getpass function

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()
HUGGINGFACEHUB_API_TOKEN

In [ ]:
from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from huggingface_hub import InferenceClient
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify HF API token is set
if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    raise ValueError("HUGGINGFACEHUB_API_TOKEN environment variable is not set")

# Define the question and template
question = "What was the first Disney movie?"
template = """Question: {question}
Answer: give a direct answer"""

# Create prompt template
prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)

class HuggingFaceHubWithTask(HuggingFaceHub):
    def _call(self, prompt: str, **kwargs) -> str:
        client = InferenceClient(
            model=self.repo_id, 
            token=self.client.token
        )
        response = client.text_generation(
            prompt,
            temperature=self.model_kwargs.get("temperature", 0.5),
            max_new_tokens=self.model_kwargs.get("max_new_tokens", 512)
        )
        # The response is already a string, no need to index it
        return response

# Initialize with Falcon model
llm = HuggingFaceHubWithTask(
    repo_id="tiiuae/falcon-7b-instruct",
    model_kwargs={
        "temperature": 0.5,
        "max_new_tokens": 512
    }
)

try:
    # Create and run the chain using the pipe syntax
    chain = prompt | llm
    response = chain.invoke({"question": question})
    print(response)
except Exception as e:
    print(f"An error occurred: {e}")